<a href="https://colab.research.google.com/github/samula98/proj3pt1v00/blob/main/Copy_of_SQL_1_Chinook_project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project SQL - Chinook


## Chinook data set

See the lecture on SQLite3 using the Chinook data set to set up the software, database, and tables, as well as for the links to ancillary information about the data set.


In [1]:
# Install the sqlite package for Ubuntu
%%capture
%%bash
apt-get update
apt-get install -y sqlite3


In [2]:
# Download the Chinook sqlite database
%%bash
[ -f chinook.zip ] ||
  curl -s -O https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
unzip -l chinook.zip


Archive:  chinook.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
   884736  2015-11-29 10:53   chinook.db
---------                     -------
   884736                     1 file


In [3]:
# unzipping file
!unzip -u chinook.zip


Archive:  chinook.zip
  inflating: chinook.db              


In [4]:
# listing out all the files
!ls -la

total 1180
drwxr-xr-x 1 root root   4096 Oct 27 21:27 .
drwxr-xr-x 1 root root   4096 Oct 27 21:26 ..
-rw-r--r-- 1 root root 884736 Nov 29  2015 chinook.db
-rw-r--r-- 1 root root 305596 Oct 27 21:27 chinook.zip
drwxr-xr-x 4 root root   4096 Oct 24 13:20 .config
drwxr-xr-x 1 root root   4096 Oct 24 13:20 sample_data


In [5]:
# List of the tables in the database
%%script sqlite3 --column --header chinook.db
.tables


albums          employees       invoices        playlists     
artists         genres          media_types     tracks        
customers       invoice_items   playlist_track


In [6]:
# Show schema for the entire database
%%script sqlite3 --column --header chinook.db
.schema


CREATE TABLE IF NOT EXISTS "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);
CREATE TABLE sqlite_sequence(name,seq);
CREATE TABLE IF NOT EXISTS "artists"
(
    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Name] NVARCHAR(120)
);
CREATE TABLE IF NOT EXISTS "customers"
(
    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) 
		ON DELETE NO ACTION ON 

In [7]:
# Count of number of unique albums
%%script sqlite3 --column --header chinook.db
select count(distinct AlbumID)
from tracks


count(distinct AlbumID)
-----------------------
347                    


In [8]:
# Count of number of countries that have records
%%script sqlite3 --column --header chinook.db
select count(distinct BillingCity)
from invoices

count(distinct BillingCity)
---------------------------
53                         


In [9]:
# Count of countries of the customers
%%script sqlite3 --column --header chinook.db
select count(distinct Country)
from customers


count(distinct Country)
-----------------------
24                     


## Come up with questions about your data
Have a look at the Entity-Relation ( ER ) diagram to help come up with questions.

* What sort of information is in this dataset?
* How many records are there?
* How many different countries (states, counties, cities, etc) have records in this data set?


If you are stuck, here are some ideas for questions:
- https://github.com/LucasMcL/15-sql_queries_02-chinook/blob/master/README.md
- [Using the R language]( https://rpubs.com/enext777/636199 )
- [Search Google]( https://www.google.com/search?q=chinook+database+questions )



In [10]:
# What sort of information is in this dataset?
# There is information about the relation between employees, customers, invoices, albums, artists,
# tracks, playlists, and genres within the ER diagram. There is also the names and ID numbers of
# the different objects within each table.

# How many records are there?
# 347 unique albums

# How many different countries (states, counties, cities, etc) have records in this data set?
# 53 unique cities from invoices
# 24 unique countries from customers

## Use SQL queries to pull specific information

Do NOT pull all the data and then filter using DataFrame methods etc. Make sure and use AT LEAST 13 of the 15 SQL options listed below. (You may have to get creative and come up with more questions to ask/answer.)


### Basic Queries


#### SELECT (with * and with column names)


In [11]:
# What are the first 10 entries from the albums table?
%%script sqlite3 --column --header chinook.db
select *
from albums
limit 10

AlbumId  Title                                  ArtistId
-------  -------------------------------------  --------
1        For Those About To Rock We Salute You  1       
2        Balls to the Wall                      2       
3        Restless and Wild                      2       
4        Let There Be Rock                      1       
5        Big Ones                               3       
6        Jagged Little Pill                     4       
7        Facelift                               5       
8        Warner 25 Anos                         6       
9        Plays Metallica By Four Cellos         7       
10       Audioslave                             8       


In [12]:
# What are the names of the countries that the customers are from?
%%script sqlite3 --column --header chinook.db
select distinct Country
from Customers

Country       
--------------
Brazil        
Germany       
Canada        
Norway        
Czech Republic
Austria       
Belgium       
Denmark       
USA           
Portugal      
France        
Finland       
Hungary       
Ireland       
Italy         
Netherlands   
Poland        
Spain         
Sweden        
United Kingdom
Australia     
Argentina     
Chile         
India         


#### WHERE


In [13]:
# What are Audioslaves album names?
%%script sqlite3 --column --header chinook.db
select *
from artists
join albums
on artists.ArtistID = albums.ArtistID
where Name = 'Audioslave'

ArtistId  Name        AlbumId  Title         ArtistId
--------  ----------  -------  ------------  --------
8         Audioslave  10       Audioslave    8       
8         Audioslave  11       Out Of Exile  8       
8         Audioslave  271      Revelations   8       


#### AND


In [14]:
# Who are the customers from Brazil with 3 as Support Rep ID?
%%script sqlite3 --column --header chinook.db
select *
from customers
where country = 'Brazil'
and SupportRepID = 3

CustomerId  FirstName  LastName   Company                                           Address                          City                 State  Country  PostalCode  Phone               Fax                 Email                          SupportRepId
----------  ---------  ---------  ------------------------------------------------  -------------------------------  -------------------  -----  -------  ----------  ------------------  ------------------  -----------------------------  ------------
1           Luís       Gonçalves  Embraer - Empresa Brasileira de Aeronáutica S.A.  Av. Brigadeiro Faria Lima, 2170  São José dos Campos  SP     Brazil   12227-000   +55 (12) 3923-5555  +55 (12) 3923-5566  luisg@embraer.com.br           3           
12          Roberto    Almeida    Riotur                                            Praça Pio X, 119                 Rio de Janeiro       RJ     Brazil   20040-020   +55 (21) 2271-7000  +55 (21) 2271-7070  roberto.almeida@riotur.gov.br  3           


#### OR


In [15]:
# Who are the customers from Argentina and Brazil?
%%script sqlite3 --column --header chinook.db
select *
from customers
where country = 'Argentina'
or country = 'Brazil'

CustomerId  FirstName  LastName   Company                                           Address                          City                 State  Country    PostalCode  Phone                Fax                 Email                          SupportRepId
----------  ---------  ---------  ------------------------------------------------  -------------------------------  -------------------  -----  ---------  ----------  -------------------  ------------------  -----------------------------  ------------
1           Luís       Gonçalves  Embraer - Empresa Brasileira de Aeronáutica S.A.  Av. Brigadeiro Faria Lima, 2170  São José dos Campos  SP     Brazil     12227-000   +55 (12) 3923-5555   +55 (12) 3923-5566  luisg@embraer.com.br           3           
10          Eduardo    Martins    Woodstock Discos                                  Rua Dr. Falcão Filho, 155        São Paulo            SP     Brazil     01007-010   +55 (11) 3033-5446   +55 (11) 3033-4564  eduardo@woodstock.com.br       4

#### LIKE (with % or _ wildcard)


In [16]:
# Who are the customers with a first name that starts with the letter A?
%%script sqlite3 --column --header chinook.db
select *
from customers
where FirstName like 'A%'

CustomerId  FirstName  LastName  Company               Address                               City       State  Country  PostalCode  Phone               Fax                 Email                   SupportRepId
----------  ---------  --------  --------------------  ------------------------------------  ---------  -----  -------  ----------  ------------------  ------------------  ----------------------  ------------
7           Astrid     Gruber                          Rotenturmstraße 4, 1010 Innere Stadt  Vienne            Austria  1010        +43 01 5134505                          astrid.gruber@apple.at  5           
11          Alexandre  Rocha     Banco do Brasil S.A.  Av. Paulista, 2022                    São Paulo  SP     Brazil   01310-200   +55 (11) 3055-3278  +55 (11) 3055-8131  alero@uol.com.br        5           
32          Aaron      Mitchell                        696 Osborne Street                    Winnipeg   MB     Canada   R3L 2B9     +1 (204) 452-6452               

In [17]:
# Who are the customers with '-y' in their name?
%%script sqlite3 --column --header chinook.db
select *
from customers
where Country like '_y'
;

#### BETWEEN


In [ ]:
# What are the InvoiceID's that are between 25 and 35 in invoice_items?
%%script sqlite3 --column --header chinook.db
select *
from invoice_items
where InvoiceID between 25 and 35

#### LIMIT



In [ ]:
# What are the first 10 items in invoice_items that have a unit price of 1.99?
%%script sqlite3 --column --header chinook.db
select *
from invoice_items
where UnitPrice = 1.99
limit 10

### Sorting and Grouping


#### ORDER BY


In [ ]:
# Order customers from Argentina and Brazil by their SupportRepID
%%script sqlite3 --column --header chinook.db
select *
from customers
where country = 'Argentina'
or country = 'Brazil'
order by SupportRepID

#### DISTINCT


In [ ]:
# How many distinct GenreID's are there in tracks?
%%script sqlite3 --column --header chinook.db
select count(distinct GenreID)
from tracks

#### GROUP BY



### Aggregates


#### MAX


#### MIN


#### SUM


#### AVG


#### COUNT



In [ ]:
# How many unique albums are there?
%%script sqlite3 --column --header chinook.db
select count(distinct AlbumID)
from tracks

In [ ]:
# How many countries are the customers from?
%%script sqlite3 --column --header chinook.db
select count(distinct Country)
from customers

## Make some plots

Make some cool plots to go with your data. Write SQL queries to get ONLY the information you need for each plot. (Don't pull ALL the data and then just plot a few columns.)



## EXTRA CREDIT:
* Use a CTE
* Use a query that joins two or more tables.
* Make a model to see if you can predict something
* Come up with something else cool to do with your data
